## Задание. 
Для заданного набора данных (по Вашему варианту) постройте модели классификации или регрессии (в зависимости от конкретной задачи, рассматриваемой в наборе данных). Для построения моделей используйте методы 1 и 2 (по варианту для Вашей группы). Оцените качество моделей на основе подходящих метрик качества (не менее двух метрик). Какие метрики качества Вы использовали и почему? Какие выводы Вы можете сделать о качестве построенных моделей? Для построения моделей необходимо выполнить требуемую предобработку данных: заполнение пропусков, кодирование категориальных признаков, и т.д.
## Условия по варианту (ИУ5-62Б Вар.14)
ИУ5-62Б, ИУ5Ц-82Б	Метод опорных векторов	Случайный лес

14. https://www.kaggle.com/noriuk/us-education-datasets-unification-project (файл states_all.csv)

## Загрузка датасета

In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Загрузка данных
df = pd.read_csv('D:/Загрузки/states_all.csv')
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1715 entries, 0 to 1714
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   PRIMARY_KEY                   1715 non-null   object 
 1   STATE                         1715 non-null   object 
 2   YEAR                          1715 non-null   int64  
 3   ENROLL                        1224 non-null   float64
 4   TOTAL_REVENUE                 1275 non-null   float64
 5   FEDERAL_REVENUE               1275 non-null   float64
 6   STATE_REVENUE                 1275 non-null   float64
 7   LOCAL_REVENUE                 1275 non-null   float64
 8   TOTAL_EXPENDITURE             1275 non-null   float64
 9   INSTRUCTION_EXPENDITURE       1275 non-null   float64
 10  SUPPORT_SERVICES_EXPENDITURE  1275 non-null   float64
 11  OTHER_EXPENDITURE             1224 non-null   float64
 12  CAPITAL_OUTLAY_EXPENDITURE    1275 non-null   float64
 13  GRA

## Предобработка данных


In [37]:
target = 'AVG_MATH_4_SCORE'

# Удалим строки, где отсутствует значение целевой переменной
df = df.dropna(subset=[target])

# Обновим X и y после удаления
features = df.drop(columns=[target])
X = features
y = df[target]

# Разделение на числовые и категориальные признаки
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Препроцессинг
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# Разделение на train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Обучение SVM

In [38]:
svm_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', SVR())
])

svm_pipeline.fit(X_train, y_train)
y_pred_svm = svm_pipeline.predict(X_test)

# Метрики
print("Метрики качества для SVM:")
print(f"MSE: {mean_squared_error(y_test, y_pred_svm):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_svm)):.4f}")
print(f"R²: {r2_score(y_test, y_pred_svm):.4f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred_svm):.4f}")


Метрики качества для SVM:
MSE: 13.1007
RMSE: 3.6195
R²: 0.8516
MAE: 2.5951


## Обучение случайный лес

In [39]:
from sklearn.ensemble import RandomForestRegressor

# Создание пайплайна для Random Forest
rf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

# Обучение модели
rf_pipeline.fit(X_train, y_train)

# Предсказания
y_pred_rf = rf_pipeline.predict(X_test)

# Метрики
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)
r2_rf = r2_score(y_test, y_pred_rf)
mae_rf = mean_absolute_error(y_test, y_pred_rf)

print("\nМетрики качества для Random Forest:")
print(f"MSE: {mse_rf:.4f}")
print(f"RMSE: {rmse_rf:.4f}")
print(f"R²: {r2_rf:.4f}")
print(f"MAE: {mae_rf:.4f}")



Метрики качества для Random Forest:
MSE: 7.2403
RMSE: 2.6908
R²: 0.9180
MAE: 2.0483


Random Forest значительно превосходит SVM по всем ключевым метрикам:

Ошибки (MSE, RMSE, MAE) меньше, что говорит о более точных предсказаниях;

Коэффициент детерминации R² выше, то есть модель лучше объясняет вариацию целевой переменной.

SVM хоть и показал неплохой результат (R² = 0.85), всё же уступает по всем показателям.

## Вывод

**Модель Random Forest демонстрирует лучшее качество предсказаний по сравнению с SVM и является предпочтительным выбором для данной задачи регрессии.**